In [ ]:
import polars as pl

# Exemple de LazyFrame avec FORMAT et VALEUR
lf = pl.LazyFrame({
    "FORMAT": ["A|B|C", "X|Y", "M|N|O|P"],
    "VALEUR": ["1|2|3", "10|20", "100|200|300|400"]
})

# Etape 1 : Splitter les colonnes en listes
lf = lf.with_columns([
    pl.col("FORMAT").str.split("|").alias("keys"),
    pl.col("VALEUR").str.split("|").alias("values")
])

# Etape 2 : Créer un dictionnaire clé -> valeur par ligne
lf = lf.with_columns(
    pl.struct(["keys", "values"]).map_elements(
        lambda row: dict(zip(row["keys"], row["values"])),
        return_dtype=pl.Struct
    ).alias("kv_dict")
)

# Etape 3 : Transformer le dictionnaire en colonnes séparées
lf = lf.unnest("kv_dict")


# Collecter et afficher le résultat
df = lf.collect()
print(df)

# Etape 4 : Renommer les colonnes
df = df.rename({col: f"sample_{col}" for col in lf.schema if col not in ["FORMAT", "VALEUR"]})

# Sauvegarde en Parquet avec les colonnes splitées
df.write_parquet("../db/variants.parquet")


shape: (3, 4)
┌─────────┬─────────────────┬───────────────────┬─────────────────────────┐
│ FORMAT  ┆ VALEUR          ┆ sample_keys       ┆ sample_values           │
│ ---     ┆ ---             ┆ ---               ┆ ---                     │
│ str     ┆ str             ┆ list[str]         ┆ list[str]               │
╞═════════╪═════════════════╪═══════════════════╪═════════════════════════╡
│ A|B|C   ┆ 1|2|3           ┆ ["A", "B", "C"]   ┆ ["1", "2", "3"]         │
│ X|Y     ┆ 10|20           ┆ ["X", "Y"]        ┆ ["10", "20"]            │
│ M|N|O|P ┆ 100|200|300|400 ┆ ["M", "N", … "P"] ┆ ["100", "200", … "400"] │
└─────────┴─────────────────┴───────────────────┴─────────────────────────┘


/tmp/ipykernel_11151/2236808875.py:29: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  [pl.col(col).alias(f"sample_{col}") if col not in ["FORMAT", "VALEUR"] else pl.col(col) for col in lf.schema]
/tmp/ipykernel_11151/2236808875.py:33: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  lf.sink_parquet("../db/variants.parquet")


In [1]:
import polars as pl

# Exemple de DataFrame
data = {
    'FORMAT': ['cle1|cle2|cle3', 'cle1|cle2', 'cle3|cle4'],
    'GENOTYPE': ['valeur1|valeur2|valeur3', 'valeur4|valeur5', 'valeur6|valeur7']
}
df = pl.LazyFrame(data)

# Fonction pour splitter et créer les colonnes
def split_and_expand(row):
    keys = row['FORMAT'].split('|')
    values = row['GENOTYPE'].split('|')
    return {key: value for key, value in zip(keys, values)}

# Appliquer la fonction à chaque ligne et créer un nouveau DataFrame
expanded_df = df.with_columns(
    pl.struct(['FORMAT', 'GENOTYPE'])
    .map_elements(
        lambda row: split_and_expand(row),
        return_dtype=pl.Struct
    )
    .alias('expanded')
)

# Extraire les noms des clés pour renommer les colonnes
# On prend la première ligne pour déterminer les noms des colonnes
first_row_keys = df['FORMAT'][0].split('|')

# Exploser la colonne 'expanded' en plusieurs colonnes avec les noms des clés
result_df = expanded_df.with_columns(
    pl.col('expanded').struct.rename_fields(first_row_keys)
).unnest('expanded')

# Afficher le résultat
print(result_df)

TypeError: 'LazyFrame' object is not subscriptable (aside from slicing)

Use `select()` or `filter()` instead.